In [ ]:
import cv2
import time
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

def count_fingers(hand_landmarks, handedness_label):
    tip_ids = [4, 8, 12, 16, 20]
    pip_ids = [3, 6, 10, 14, 18]  

    lm = hand_landmarks.landmark
    fingers = []

    if handedness_label == "Right":
        fingers.append(lm[tip_ids[0]].x > lm[pip_ids[0]].x)
    else:
        fingers.append(lm[tip_ids[0]].x < lm[pip_ids[0]].x)

    for i in range(1, 5):
        fingers.append(lm[tip_ids[i]].y < lm[pip_ids[i]].y)

    return sum(fingers)

def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ ERROR: Could not open webcam. Try changing index: VideoCapture(1) or (2).")
        return

    prev_time = time.time()

    with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
        max_num_hands=1
    ) as hands:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or frame is None:
                print("⚠️ Warning: Empty frame received. Skipping...")
                continue

            frame = cv2.flip(frame, 1)

            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rgb.flags.writeable = False
            results = hands.process(rgb)

            finger_count = 0

            if results.multi_hand_landmarks:
                handedness_label = "Right"
                if results.multi_handedness and len(results.multi_handedness) > 0:
                    handedness_label = results.multi_handedness[0].classification[0].label

                hand_landmarks = results.multi_hand_landmarks[0]

                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS
                )

                finger_count = count_fingers(hand_landmarks, handedness_label)

                cv2.putText(frame, f"Hand: {handedness_label}", (10, 80),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            now = time.time()
            fps = 1.0 / max(now - prev_time, 1e-6)
            prev_time = now

            cv2.putText(frame, f"Fingers: {finger_count}", (10, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
            cv2.putText(frame, f"FPS: {fps:.1f}", (10, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            cv2.imshow("Finger Counter", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

